In [28]:
# import libraries needed
import pandas as pd

pd.set_option("display.max_columns", None)
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, skew, probplot
import warnings

In [29]:
#read in the data file
track_df = pd.read_csv("dataset.csv")

In [30]:
#rows and columns of table
track_df.shape

(114000, 21)

In [31]:
#number of categorical variables
cat_num = 0
for dtype in track_df.dtypes:
    if dtype == "object":
        cat_num = cat_num + 1

In [32]:
print("# of categorical variables:", cat_num)

#rest of the columns are numeric
numeric_vars = track_df.shape[1] - cat_num - 2 #subtract 2 columns for number and track id columns
print("# of numeric variables:", numeric_vars) 

# of categorical variables: 5
# of numeric variables: 14
